In [1]:
import pandas as pd
from pathlib import Path
import os
from dotenv import load_dotenv
import geopandas as gpd

In [ ]:

import joblib
import numpy as np
import pandas as pd


In [ ]:
BASE_DIR = Path(os.getenv("BASE_DIR"))

In [ ]:


BASE_DIR = Path(BASE_DIR /"Train_Final2")
RUN_DIR  = BASE_DIR / "ml_runs_pu_ensemble_xgb"
SETTING  = "all_bg"   # setting tốt nhất của bạn

MODELS_DIR = RUN_DIR / SETTING / "models"
FEAT_PATH  = BASE_DIR / "feature_cols.txt"
NEW_DATA = BASE_DIR / "Main output" / "whole_NT_grid_features.parquet"
OUT_FILE = BASE_DIR / "new_data_pred_ensemble.parquet"


In [ ]:

feature_cols = pd.read_csv(
    FEAT_PATH,
    header=None
)[0].tolist()
df = pd.read_parquet(NEW_DATA)

missing = [c for c in feature_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing {len(missing)} feature columns, e.g.: {missing[:10]}")

X = df[feature_cols].to_numpy(dtype=np.float32, copy=False)

In [ ]:

model_files = sorted(MODELS_DIR.glob("model_repeat_*.joblib"))
if not model_files:
    raise RuntimeError(f"No models found in {MODELS_DIR}")

print(f"Loaded {len(model_files)} models")

preds = []
for fp in model_files:
    m = joblib.load(fp)
    preds.append(m.predict_proba(X)[:, 1].astype(np.float32))

P = np.vstack(preds)   # shape: (n_models, n_samples)

pred_mean = P.mean(axis=0).astype(np.float32)
pred_std  = P.std(axis=0).astype(np.float32)


out = df.copy()
out["pred_mean"] = pred_mean
out["pred_std"]  = pred_std

out.to_parquet(OUT_FILE, index=False)

print("Saved:", OUT_FILE)
print(out[["pred_mean", "pred_std"]].describe())


Loaded 1 models


C:\Users\Phong\Desktop\GIS\Project 2\gisenv\lib\site-packages\xgboost\core.py:774: UserWarning: [10:57:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


Saved: C:\Users\Phong\Desktop\GIS\Project 2\Train_Final2\new_data_pred_ensemble.parquet
          pred_mean   pred_std
count  6.524352e+06  6524352.0
mean   1.564624e-01        0.0
std    2.588863e-01        0.0
min    1.686108e-07        0.0
25%    1.972232e-03        0.0
50%    2.603022e-02        0.0
75%    1.753712e-01        0.0
max    9.996802e-01        0.0
